In [17]:
import numpy as np
import glob
import matplotlib.pyplot as plt
import cv2
from skimage import measure, segmentation
from vis_utils import load_volume, VolumeVisualizer
from scipy.ndimage import zoom
from skimage.morphology import skeletonize, skeletonize_3d

In [34]:
source_dir = './data/'
files = glob.glob(source_dir + '/*/*.raw')
files[0]

'./data/P320/320_60um_1739x553x960.raw'

In [35]:
%%time
sample_volume = load_volume(files[1], scale=0.4)
print(sample_volume.shape)

(460, 371, 691)
CPU times: user 2.01 s, sys: 6.93 s, total: 8.95 s
Wall time: 12.8 s


In [5]:
visualizer = VolumeVisualizer(sample_volume).visualize(scale=0.5)

In [4]:
binary_mask = (sample_volume > 90).astype(np.uint8)
binary_mask.shape

(384, 221, 696)

In [5]:
visualizer = VolumeVisualizer(binary_mask).visualize(scale=1, primary_color=(255,255,255))

In [26]:
def get_main_regions(binary_mask, min_size=10_000, connectivity=3):
    labeled = measure.label(binary_mask, connectivity=connectivity)
    region_props = measure.regionprops(labeled)
    
    main_regions_masks = []
    
    for props in region_props:
        if props.area >= min_size:
            main_regions_masks.append((props.filled_image, props.bbox))
            
    return main_regions_masks

def merge_masks(masks, img_shape):
    result_mask = np.zeros(img_shape, dtype=np.uint8)
    for mask, bbox in masks:
        min1, min2, min3, max1, max2, max3 = bbox
        result_mask[min1:max1, min2:max2, min3:max3] += mask.astype(np.uint8)
        
    return result_mask

In [7]:
%%time
main_regions_masks = get_main_regions(binary_mask, min_size=5_000, connectivity=1)
len(main_regions_masks)

CPU times: user 10.6 s, sys: 479 ms, total: 11.1 s
Wall time: 11.1 s


2

In [8]:
visualizer = VolumeVisualizer(
    main_regions_masks[0][0]
).visualize(scale=1, primary_color=(255,255,255))

In [33]:
merged_mask = merge_masks(main_regions_masks, binary_mask.shape)
merged_mask[50:180, 50:160, 70:200] = 0.5

visualizer = VolumeVisualizer(
    merged_mask * 100
).visualize(scale=1, primary_color=(255,255,255))

In [16]:
main_regions_masks[0][0].max()

True

## morphological skeletonization central line extraction

In [25]:
%%time

sample_mask = merge_masks(main_regions_masks, binary_mask.shape)

skeleton = skeletonize_3d(sample_mask)

(384, 221, 696)
(29, 45, 173, 384, 193, 661)
(37, 25, 23, 384, 163, 389)
CPU times: user 3.84 s, sys: 131 ms, total: 3.97 s
Wall time: 3.98 s


In [27]:
visualizer = VolumeVisualizer(
    skeleton
).visualize(scale=1, primary_color=(255,255,255))